In [1]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

In [3]:
H=nx.read_gml('gml_topo/Bren.gml',  label='id')
H

In [4]:
# Base for measure is Mb/s
mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}

In [5]:
def unit_measure(strr):
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

In [6]:
bflows = {}
for e in H.edges():
    data = H.get_edge_data(e[1], e[0])
    if data['LinkLabel'] != 'Ethernet':
        bandwidth = unit_measure(data['LinkLabel'])
        step = bandwidth / 100.0
        back_flow = random.randrange(0.0, bandwidth, step)
        bflows[e] = {'BackgroundFlow': back_flow}
        
nx.set_edge_attributes(H, bflows)
# print("New data in network")
# print(H.edges(data = True))

In [7]:
# p_capacity - матрица пропускных способностей

p_capacity = [[   0, 110,   0,   0,  60],
			  [ 110,   0, 150, 100,   0],
			  [   0, 150,   0, 130,   0],
			  [   0, 100, 130,   0, 120],
			  [  60,   0,   0, 120,   0]]

# f_flows - матрица фоновых потоков

f_flows =  [[  0,  10,   0,   0,  20],
			[ 10,   0,  20,  70,   0],
			[  0,  20,   0,  10,   0],
			[  0,  70,  10,   0,  10],
			[ 20,   0,   0,  10,   0]]

In [8]:
def print_pGraph(p_capacity):
    nodes = []
    for node in range(len(p_capacity)):
        nodes.append({'id': node, 'label': node})
    edges = []
    for i in range(len(p_capacity)):
        for j in range(i, len(p_capacity)):
            if p_capacity[i][j] != 0:
                perc = p_capacity[i][j]
                edges.append({'from': i, 'to': j, 
                              'dashes':False, 'label': (str)(p_capacity[i][j]) + 'Mb/s', 
                              'font':{'align': 'top'},
                              'color': 'BLUE',
                              'width': 4})
    return (nodes, edges)

In [9]:
def print_fGraph(f_flows):
    nodes = []
    for node in range(len(f_flows)):
        nodes.append({'id': node, 'label': node})
    edges = []
    for i in range(len(f_flows)):
        for j in range(i, len(f_flows)):
            if f_flows[i][j] != 0:
                perc = f_flows[i][j]
                edges.append({'from': i, 'to': j, 
                              'dashes':False, 'label': (str)(f_flows[i][j]) + 'Mb/s', 
                              'font':{'align': 'top'},
                              'color': 'BLUE',
                              'width': 4})
    return (nodes, edges)

In [10]:
def print_Graph(p_capacity, f_flows):
    nodes = []
    for node in range(len(p_capacity)):
        nodes.append({'id': node, 'label': node})
    edges = []
    for i in range(len(p_capacity)):
        for j in range(i, len(p_capacity)):
            if p_capacity[i][j] != 0:
                perc = f_flows[i][j] / p_capacity[i][j]
                color = 'GREEN'  
                if perc <= 0.5:
                    color = 'GREEN'
                elif perc <= 0.8:
                    color = 'YELLOW'
                elif perc <= 1:
                    color = 'RED'
                edges.append({'from': i, 'to': j, 
                              'dashes':False, 'label': (str)(p_capacity[i][j] - f_flows[i][j]) + 'Mb/s', 
                              'font':{'align': 'top'},
                              'color': color,
                              'width': 4})
    return (nodes, edges)

In [11]:
nodes, edges = print_Graph(p_capacity, f_flows)
#nodes, edges = print_pGraph(p_capacity)
#nodes, edges = print_fGraph(f_flows)

In [12]:
# Transform the graph into a JSON graph
data = {"nodes":nodes, "edges":edges}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [13]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    
    var options = {
        width: '900px',
        height: '500px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [14]:
%%html
<div id="mynetwork"></div>

In [15]:
Bandwidth = pd.DataFrame(p_capacity)
BackFlows = pd.DataFrame(f_flows)

In [16]:
BackFlows.head(5)

,0,1,2,3,4
0,0,10,0,0,20
1,10,0,20,70,0
2,0,20,0,10,0
3,0,70,10,0,10
4,20,0,0,10,0


In [17]:
from itertools import product

def print_answer(correct, answ, V, K, M):
    if correct == "correct":
        for i, j, k, t in product(V, V, K, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)

In [18]:
Contracts = [{'volume': 600, 't_s': 0, 't':  100, 'A': 0, 'B': 4}, 
             {'volume': 100, 't_s': 10, 't':  50, 'A': 1, 'B': 2}]

In [19]:
def set_time(t0, T, Contracts):
    t = {}
    points = set()
    points.add(t0)
    points.add(T)
    for c in Contracts:
        points.add(c['t_s'])
        points.add(c['t_s'] + c['t'])
    points = sorted(list(points))
    for i in range(len(points) - 1):
        e = points[i + 1] - points[i]
        C = set()
        for c in range(len(Contracts)):
            if points[i] >= Contracts[c]['t_s'] and points[i] < Contracts[c]['t_s'] + Contracts[c]['t']:
                C.add(c)
        t[i] = {'em': e, 'Ctm': C}
    return points, t

In [20]:
V = set(range(Bandwidth.shape[0])) # число вершин
K = set(range(len(Contracts)))     # количество контрактов К
t0 = 0                             # начало отсчета t0
T = 1000                           # конец отсчета в сек
timeline, Intervals = set_time(t0, T, Contracts)
M = set(range(len(Intervals)))
timeline, Intervals

([0, 10, 60, 100, 1000],
 {0: {'em': 10, 'Ctm': {0}},
  1: {'em': 50, 'Ctm': {0, 1}},
  2: {'em': 40, 'Ctm': {0}},
  3: {'em': 900, 'Ctm': set()}})

In [21]:
import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

Using Python-MIP package version 1.6.8


In [22]:
correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(correct_pyomo, answ_pyomo, V, K, M)

Total time:  0.42928504943847656
answer[0][4][0][0] = 6.0
answer[0][4][0][1] = 6.0
answer[0][4][0][2] = 6.0
answer[1][2][1][1] = 2.0


In [23]:
def send_toJS(t, K, p_capacity, f_flows, answ):
    nodes = []
    for node in range(len(p_capacity)):
        nodes.append({'id': node, 'label': node})
    edges = []
    for i in range(len(p_capacity)):
        for j in range(i, len(p_capacity)):
            if p_capacity[i][j] != 0:
                k_flow = 0
                label = ''
                for k in K:
                    if answ[i][j][k][t] != 0:
                        k_flow += answ[i][j][k][t]
                        label += ', №' + (str)(k)
                    if answ[j][i][k][t] != 0:
                        k_flow += answ[j][i][k][t]
                        label += ', №' + (str)(k)
                color = 'BLUE'
                if k_flow != 0:
                    perc = (k_flow + f_flows[i][j]) / p_capacity[i][j]
                    label += "\n" + (str)(k_flow) + "Mb/s"
                    if perc <= 0.5:
                        color = 'GREEN'
                    elif perc <= 0.8:
                        color = 'YELLOW'
                    elif perc <= 1:
                        color = 'RED'        
                edges.append({'from': i, 'to': j, 
                              'dashes':False, 'label': label[2:],
                              'font':{'align': 'top'},
                              'color': color,
                              'width': 4})
            
    # Transform the graph into a JSON graph
    data = {"nodes":nodes, "edges":edges}
    jsonGraph = json.dumps(data, indent=4)

    return jsonGraph

In [24]:
t = 0
jsonGraph = send_toJS(t, K, p_capacity, f_flows, answ_pyomo)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [25]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_pyomo');
    
    var options = {
        width: '1000px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [26]:
%%html
<div id="resultnetwork_pyomo"></div>

In [ ]:
correct_mip, answ_mip, time_mip = mip.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(correct_mip, answ_mip, V, K, M)

In [ ]:
(answ_mip == answ_pyomo).all()

In [ ]:
t = 0
jsonGraph = send_toJS(t, K, p_capacity, f_flows, answ_mip)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

In [ ]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_mip');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

In [ ]:
%%html
<div id="resultnetwork_mip"></div>